# Byggingagreining á gervitunglamyndum
Nathan HK

In [1]:
from io import BytesIO
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pyrosm import OSM
from pyrosm import get_data
import pywikibot
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from shapely.geometry import Point
import sklearn as sk
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
mappa = '/Users/002-nathan/Desktop/Envalys/gtm/'
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

## Inngangsorð
Við viljum þjálfa gervigreindarlíkan til að greina byggingar á gervitunglamyndum. Við þurfum tvenn gögn: gervitunglamyndir og staðsetningar bygginga.
- **Gervitunglamyndir:** Við notum skjámyndatökur af Já.is. Ég veit ekki hvort þetta sé löglegt, en ég er ekki með neinar betri leiðir.
- **Staðsetningar bygginga:** Við notum OpenStreetMap.

Ég nota Apple M1 Pro-örgjörvi með 16 GB minni.

## Gervitunglamyndir
Við notum lista yfir greinar á ensku Wikipediunni um staði á Höfuðborgarsvæðinu og Akureyri. Gögnin á Landsbyggðinni eru ekki nóg nákvæm fyrir þetta líkan.

In [3]:
byrjun = time.time()
hnitlisti = []
wiki = pywikibot.Site('en', 'wikipedia')

# Capital Region
flokkur_h = pywikibot.Category(wiki, 'Populated places in Capital Region (Iceland)')
undir = list(flokkur_h.subcategories(recurse=3))
buinn_fl = []  # Completed categories
buinn_si = []  # Completed pages
print(len(undir))
for a in undir:
    if a in buinn_fl:  # Already did category
        continue
    for b in a.articles():
        if b in buinn_si:  # Already did page
            continue
        coord = b.coordinates(primary_only=True)
        # Coordinates are invalid outside latitude [63, 67] and longitude [-25, -12]
        if coord is not None and coord.lat >= 63 and coord.lat <= 67 and coord.lon >= -25 and coord.lon <= -12:
            hnitlisti.append(('h', float(coord.lat), float(coord.lon)))
        buinn_si.append(b)
    buinn_fl.append(a)
print('h', len(hnitlisti), time.time() - byrjun)

# Akureyri
flokkur_a = pywikibot.Category(wiki, 'Akureyri')
undir = list(flokkur_a.subcategories(recurse=3))
buinn_fl = []  # Completed categories
buinn_si = []  # Completed pages
print(len(undir))
for a in undir:
    if a in buinn_fl:  # Already did category
        continue
    for b in a.articles():
        if b in buinn_si:  # Already did page
            continue
        coord = b.coordinates(primary_only=True)
        # Coordinates are invalid outside latitude [63, 67] and longitude [-25, -12]
        if coord is not None and coord.lat >= 63 and coord.lat <= 67 and coord.lon >= -25 and coord.lon <= -12:
            hnitlisti.append(('a', float(coord.lat), float(coord.lon)))
        buinn_si.append(b)
    buinn_fl.append(a)
print('a', len(hnitlisti), time.time() - byrjun)

54
h 120 101.81101202964783
6
a 130 117.08099031448364


Við getum ekki tekið myndir af miðjunni skjásins, því það eru önnur HTML-efni sem hylja gervitunglamyndirnar. Þess vegna leitum við að staði sem eru 0.002° til austurs frá myndatökustaðnum; hérna eru hnitin á skjánum fyrir þennan stað.

In [4]:
skhn = (746, 861)

Við tökum skjámyndir af öllum stöðum á hnitlistanum.
- URL-ið notar ISN93-hnit, en okkar hnit eru WGS84, og það er engin einföld leið til að skipta milli þeirra. Þess vegna þurfum við að leita að hnitum eins og manneskja myndi leita.
- Á Chrome er myndasvæðið 512x512, en þegar myndin er vistuð verður hún 1024x1024.

In [5]:
byrjun = time.time()
driver = webdriver.Chrome()
driver.set_window_size(1500, 1000)
driver.get('https://ja.is/kort/?x=356954&y=408253&nz=17.00&type=aerialnl')
# Accept GDPR
try:
    btn = driver.find_element(By.XPATH, '//a[@id="gdpr_banner_ok"]')
    btn.click()
except NoSuchElementException:
    pass
# Allow cookies
try:
    btn = driver.find_element(By.XPATH, '//button[@class="ch2-btn ch2-allow-all-btn ch2-btn-primary"]')
    btn.click()
except NoSuchElementException:
    pass
leit = driver.find_element(By.XPATH, '//input[@id="mapq"]')
for n in range(len(hnitlisti)):
    if n % 50 == 0:
        print(n, time.time() - byrjun)
    hnit = hnitlisti[n]
    # Input search term into search box
    try:
        z = open(mappa + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png', 'rb')
        z.close()
    except FileNotFoundError:
        leit.clear()
        leit.send_keys(str(hnit[1]) + ', ' + str(hnit[2] + 0.002))
        leit.send_keys(Keys.RETURN)
        time.sleep(2) # Wait for images to load
        try:  # Place not found
            nf = driver.find_element(By.XPATH, '//div[@class="row not-found"]')
        except NoSuchElementException:  # Place found, save and crop screenshot
            driver.save_screenshot(mappa + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
            skmynd = Image.open(mappa + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
            skmynd = skmynd.crop((skhn[0] - 512, skhn[1] - 512, skhn[0] + 512, skhn[1] + 512))
            skmynd.save(mappa + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
        time.sleep(1)
driver.close()
print(time.time() - byrjun)

0 12.286180973052979
50 12.288287162780762
100 12.289857149124146
12.414113998413086


## Staðsetningar bygginga
Við sækjum gögn frá OpenStreetMap.

In [6]:
fp = get_data('Iceland')

Þetta undirforrit tekur díl á myndinni og finnur GPS-hnitin. Við notum Web Mercator.

In [7]:
def pix2coord(pix, hnit_br):
    x_t = hnit_br[1] + (pix[0] - 512) / 1024
    lon = math.degrees(x_t * 2 * math.pi / (2 ** 17) - math.pi)
    y_t = hnit_br[0] + (pix[1] - 512) / 1024
    lat = math.degrees(2 * (math.atan(math.exp(math.pi - y_t * 2 * math.pi / (2 ** 17))) - math.pi / 4))
    return (lat, lon)

Við sækjum tvo lista: einn yfir byggingar á Höfuðborgarsvæðinu, og einn á Akureyri.

In [8]:
byrjun = time.time()
byg_listi = {}
osm_h = OSM(fp, bounding_box=[-22.140901, 63.847886, -21.152576, 64.390306])
byg_listi['h'] = osm_h.get_buildings()
osm_a = OSM(fp, bounding_box=[-18.398071, 65.543087, -17.968359, 66.576398])
byg_listi['a'] = osm_a.get_buildings()
print(time.time() - byrjun)

24.004565954208374


Fyrir hverja skjámynd búum við til nýja mynd sem sýnir staðsetningar bygginga. Dílar með byggingum eru rauðir, og dílar án bygginga eru bláir.

Til að gera þetta fljótari:
- Við útreiknum utanferninginn (e. *bounding box*) fyrir hverja byggingu áður en við skoðum myndina. Ef díll er ekki í utanferningnum, þá er hann ekki í byggingunni, og við vitum það án þess að gera ```.contains()```, sem tekur langan tíma.
- Við búum til fjögur ílát: eitt fyrir byggingar sem eru norðvestur af miðpunktnum, eitt fyrir norðaustur, o.s.fv. Fyrir hverja mynd setjum við byggingarnar í viðeigandi ílátið, og fyrir hvern díl þurfum við bara að leita þar.

In [9]:
byrjun = time.time()
X_gogn = []
y_gogn = []
bd_all = {}
for st in ['h', 'a']:
    byggingar = byg_listi[st]
    bd = []
    for k in range(byggingar.shape[0]):
        bns = byggingar['geometry'][k].bounds
        bd.append(bns)
    bd_all[st] = bd
for n in range(len(hnitlisti)):
    if n % 10 == 0:
        print(n, time.time() - byrjun)
    hnit = hnitlisti[n]
    # Open image
    try:
        gtm = Image.open(mappa + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
        dilar = gtm.load()
    except FileNotFoundError:
        continue
    X_gogn.append(torch.tensor(np.transpose(np.array(gtm.getdata()).reshape(1024, 1024, 3), 
                                            (2, 0, 1)).reshape(1, 3, 1024, 1024), dtype=torch.float32).to(device))
    # Convert coordinates
    y_n = 1 / (2 * math.pi) * 2 ** 17 * (math.pi - math.log(math.tan(math.pi / 4 + math.radians(hnit[1]) / 2))) - 0.5
    y_s = 1 / (2 * math.pi) * 2 ** 17 * (math.pi - math.log(math.tan(math.pi / 4 + math.radians(hnit[1]) / 2))) + 0.5
    byggingar = byg_listi[hnit[0]]
    hnit_br = (1 / (2 * math.pi) * 2 ** 17 * (math.pi - math.log(math.tan(math.pi / 4 + math.radians(hnit[1]) / 2))),
               1 / (2 * math.pi) * 2 ** 17 * (math.pi + math.radians(hnit[2])))
    # Buildings
    bd = bd_all[hnit[0]]
    ilat = [[[], []], [[], []]]
    for k in range(byggingar.shape[0]):
        bns = bd[k]
        if bns[0] <= hnit[2] and bns[1] <= hnit[1] and bns[2] >= hnit[2] - 0.0014 and bns[3] >= hnit[1] - 0.0008:
            ilat[0][0].append(k)
        if bns[0] <= hnit[2] and bns[3] >= hnit[1] and bns[2] >= hnit[2] - 0.0014 and bns[1] <= hnit[1] + 0.0008:
            ilat[0][1].append(k)
        if bns[2] >= hnit[2] and bns[1] <= hnit[1] and bns[0] <= hnit[2] + 0.0014 and bns[3] >= hnit[1] - 0.0008:
            ilat[1][0].append(k)
        if bns[2] >= hnit[2] and bns[3] >= hnit[1] and bns[0] <= hnit[2] + 0.0014 and bns[1] <= hnit[1] + 0.0008:
            ilat[1][1].append(k)
    # Loop thru pixels
    y_mynd = np.zeros((1, 1, 1024, 1024))
    for i in range(1024):
        for j in range(1024):
            coord = pix2coord((i, j), hnit_br)
            f = True
            ilat_pos = [1, 1]
            if coord[1] < hnit[2]:
                ilat_pos[0] = 0
            if coord[0] < hnit[1]:
                ilat_pos[1] = 0
            for k in ilat[ilat_pos[0]][ilat_pos[1]]:
                if bd[k][0] > coord[1] or bd[k][1] > coord[0] or bd[k][2] < coord[1] or bd[k][3] < coord[0]:
                    continue
                if byggingar['geometry'][k].contains(Point(coord[1], coord[0])):
                    y_mynd[0][0][i][j] = 1
                    gtm.putpixel((i, j), (max(dilar[i,j][0] + 64, 255), dilar[i,j][1], dilar[i,j][2]))
                    f = False
                    break
            if f:
                gtm.putpixel((i, j), (dilar[i,j][0], dilar[i,j][1], max(dilar[i,j][2] + 64, 255)))
    gtm.save(mappa + 'saman_' + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
    y_gogn.append(torch.tensor(y_mynd, dtype=torch.float32).to(device))
print(time.time() - byrjun)

0 0.45939111709594727
10 90.74268698692322
20 180.24201703071594
30 289.6506083011627
40 422.8608021736145
50 507.9452350139618
60 607.4609222412109
70 676.5264420509338
80 738.953547000885
90 869.2726452350616
100 1030.293787240982
110 1115.8231172561646
120 1228.821366071701
1298.5642020702362


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_gogn, y_gogn)

## Líkan
Hérna er U-Net til að greina díla á gervitunglamyndum. Forritað með aðstoð frá o1-preview eftir OpenAI.

In [11]:
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()

        # Encoding path
        self.enc1 = self.contract_block(in_channels, 64, 7, 3)
        self.enc2 = self.contract_block(64, 128, 3, 1)
        self.enc3 = self.contract_block(128, 256, 3, 1)
        self.enc4 = self.contract_block(256, 512, 3, 1)
        self.enc5 = self.contract_block(512, 1024, 3, 1)

        # Decoding path
        self.dec5 = self.expand_block(1024, 512, 3, 1)
        self.dec4 = self.expand_block(1024, 256, 3, 1)
        self.dec3 = self.expand_block(512, 128, 3, 1)
        self.dec2 = self.expand_block(256, 64, 3, 1)
        self.dec1 = self.final_block(128, out_channels, 3, 1)

    def __call__(self, x):
        # Encoding path
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, kernel_size=2))
        enc3 = self.enc3(F.max_pool2d(enc2, kernel_size=2))
        enc4 = self.enc4(F.max_pool2d(enc3, kernel_size=2))
        enc5 = self.enc5(F.max_pool2d(enc4, kernel_size=2))

        # Decoding path
        dec5 = self.dec5(F.interpolate(enc5, scale_factor=2, mode='bilinear', align_corners=True))
        dec5 = torch.cat((enc4, dec5), dim=1)
        dec4 = self.dec4(F.interpolate(dec5, scale_factor=2, mode='bilinear', align_corners=True))
        dec4 = torch.cat((enc3, dec4), dim=1)
        dec3 = self.dec3(F.interpolate(dec4, scale_factor=2, mode='bilinear', align_corners=True))
        dec3 = torch.cat((enc2, dec3), dim=1)
        dec2 = self.dec2(F.interpolate(dec3, scale_factor=2, mode='bilinear', align_corners=True))
        dec2 = torch.cat((enc1, dec2), dim=1)
        dec1 = self.dec1(dec2)

        return dec1  # Output logits

    def contract_block(self, in_channels, out_channels, kernel_size, padding):
        contract = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
        )
        return contract

    def expand_block(self, in_channels, out_channels, kernel_size, padding):
        expand = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
        )
        return expand

    def final_block(self, in_channels, out_channels, kernel_size, padding):
        final = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.Sigmoid()  # Use sigmoid activation for binary classification
        )
        return final

In [12]:
epochs = 10

In [13]:
byrjun = time.time()
likan = UNet(in_channels=3, out_channels=1)
likan.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(likan.parameters(), lr=1e-4)
for e in range(epochs):
    for i in range(len(X_train)):
        if i % 10 == 0:
            print(e, i, time.time() - byrjun)
        y_pred = likan(X_train[i])
        loss = criterion(y_pred, y_train[i])
        loss.backward()
        optimizer.step()
        torch.mps.synchronize()
print(time.time() - byrjun)

0 0 0.4844186305999756
0 10 85.86574983596802
0 20 171.4545407295227
0 30 256.69369983673096
0 40 341.07189083099365
0 50 423.2439889907837
0 60 505.59316301345825
0 70 586.6096568107605
0 80 664.9094047546387
0 90 742.4108748435974
1 0 796.5213987827301
1 10 872.9328939914703
1 20 951.7279448509216
1 30 1028.3762538433075
1 40 1106.6773037910461
1 50 1183.9699819087982
1 60 1262.535770893097
1 70 1338.8658318519592
1 80 1416.5565297603607
1 90 1496.131395816803
2 0 1555.067566871643
2 10 1634.918063879013
2 20 1721.6563210487366
2 30 1808.876669883728
2 40 1898.9586358070374
2 50 1988.612432718277
2 60 2075.290905714035
2 70 2155.8068628311157
2 80 2233.5561039447784
2 90 2310.5302789211273
3 0 2364.3836698532104
3 10 2442.4994218349457
3 20 2523.5904738903046
3 30 2606.0000488758087
3 40 2687.017163038254
3 50 2765.3175768852234
3 60 2841.5627048015594
3 70 2919.2798359394073
3 80 2997.7056069374084
3 90 3080.6805357933044
4 0 3139.168612718582
4 10 3218.6049468517303
4 20 3297.34452

## Mat

In [14]:
thresh = 0.5

In [18]:
byrjun = time.time()
tp = 0
fp = 0
fn = 0
for i in range(len(X_train)):
    if i % 10 == 0:
        print(i, time.time() - byrjun)
    y_pred = likan(X_train[i])
    frum = y_train[i].byte()
    nid = (y_pred >= thresh).byte()
    tp += ((nid == 1) & (frum == 1)).sum().item()
    fp += ((nid == 1) & (frum == 0)).sum().item()
    fn += ((nid == 0) & (frum == 1)).sum().item()
print(time.time() - byrjun)
print('Train precision:', tp / (tp + fp))
print('Train recall:', tp / (tp + fn))
print()
tp = 0
fp = 0
fn = 0
for i in range(len(X_test)):
    if i % 10 == 0:
        print(i, time.time() - byrjun)
    y_pred = likan(X_test[i])
    frum = y_test[i].byte()
    nid = (y_pred >= thresh).byte()
    tp += ((nid == 1) & (frum == 1)).sum().item()
    fp += ((nid == 1) & (frum == 0)).sum().item()
    fn += ((nid == 0) & (frum == 1)).sum().item()
print(time.time() - byrjun)
print('Test precision:', tp / (tp + fp))
print('Test recall:', tp / (tp + fn))

0 0.00034999847412109375
10 71.12630820274353
20 148.1229109764099
30 219.04905724525452
40 285.5056369304657
50 348.4781219959259
60 416.1436092853546
70 472.2049980163574
80 543.4083652496338
90 612.8193790912628
662.7884039878845
Train precision: 0.22777357943268908
Train recall: 0.006611337787642104

0 662.7974820137024
10 723.0798711776733
20 786.327733039856
30 865.728324174881
883.9550840854645
Test precision: 0.1999940730203888
Test recall: 0.004453968160522897


## Lokaorð
Greiningin gekk illa. Hún er alls ekki nógu nákvæm fyrir eiginlega notkun. En að minnsta kosti eru prófgögnin ekki langt verri en þjálfunargögnin.

Hvert förum við héðan?
- Við reynum að greina marghyrninga heldur en díla. Við getum búið til greypingu (e. *embedding*) fyrir hverja byggingu.
- Við fáum betri gervitunglamyndir. Á þeim sem við notum núna eru byggingarnar hallaðar, og hnitin eru oft ekki nógu nákvæm.